In [1]:
!pip install matplotlib

You should consider upgrading via the 'C:\Users\adria\Desktop\hackathon\aplikacja\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import cv2
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
def predict_face(reshaped_face, model):
    CATEGORIES = ['real', 'fake']
    prediction = model.predict(reshaped_face)
    return CATEGORIES[int(prediction[0][0])]

In [4]:
def reshape_face(face):
    IMG_SIZE = 100
    img_array = cv2.imread(face) 
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

In [5]:
def get_face(image):
    file_name = 'haarcascade_frontalface_alt.xml'
    lib_dir_path = r"C:\Users\adria\Desktop\hackathon\aplikacja\venv\Lib\site-packages\cv2\data"
    face_cascade_path  = os.path.join(lib_dir_path, file_name)
    face_cascade = cv2.CascadeClassifier(face_cascade_path)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    face_exist = False
    for (x,y,w,h) in faces:
        face_exist = True
        image = image[y:y+h, x:x+w]
    if face_exist:
        return image
    else:
        return None

In [6]:
def predict_video(source_video_path, model):
    fake_frames = 0
    real_frames = 0
    vidcap = cv2.VideoCapture(source_video_path)
    success, image = vidcap.read()
    while success:
        ret, frame = vidcap.read()
        if frame is not None:
            face = get_face(frame)
#             print(type(face))
            if face is not None:
                cv2.imwrite('temporary.png', face)
#                 plt.imshow(face)
#                 plt.show()
                reshaped_face = reshape_face('temporary.png')
                result = predict_face(reshaped_face, model)
                if result == "fake":
                    fake_frames += 1
                else:
                    real_frames += 1
        else: 
            break
    print("To jest fake na {}%".format(fake_frames/(fake_frames+real_frames)))

In [9]:
model = tf.keras.models.load_model('mesonet.model')
source_video_path = r"C:\Users\adria\Desktop\hackathon\fakeExample2.mp4"
predict_video(source_video_path, model)

To jest fake na 0.5596330275229358%


In [10]:
source_video_path = r"C:\Users\adria\Desktop\hackathon\realExample2.mp4"
predict_video(source_video_path, model)

To jest fake na 0.0%
